# Genetic Test Expression from OMOP to IDEA

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

In [ ]:
vocabs = {
    "gene_expression_analysis_performed": 44808037,
    "gene_mutation_analysis_performed": 4038339,
    "tests_for_chromosome_translocations_performed": 4039364,
    "ngs_performed": 40485067,
    "pcr_test_performed": 40482792,
    "immunohistochemistry_performed": 4035726,
    "ct_dna_performed": 19387013,
}

columns = {
    "gene_expression_analysis_performed": "date_of_gene_expression",
    "gene_mutation_analysis_performed": "date_of_gene_mutation",
    "tests_for_chromosome_translocations_performed": "date_of_translocation",
    "ngs_performed": "date_of_ngs",
    "pcr_test_performed": "date_of_pcr",
    "immunohistochemistry_performed": "date_of_immunohistochemistry",
    "ct_dna_performed": "date_of_ct_dna"
}

selectProcOcurr = """SELECT person_id, procedure_date FROM procedure_occurrence 
    INNER JOIN episode_event ON procedure_occurrence.event_id = episode_event.event_id
    WHERE procedure_concept_id=%s AND episode_id=%s"""

def insert(colName,value,date,dateCol,idLine):
    updt="UPDATE genetic_test_expression SET {colName} = %s AND {dateCol} = %s WHERE id=%s"
    curIDEA.execute(updt,(value,date,idLine))
    conn2.commit()
selectDatePatientEpisode = """SELECT person_id, episode_start_date FROM episode WHERE episode_id=%s"""
selectIDEAEpisodes = """SELECT id FROM cancer_episode WHERE patient=%s AND date_of_diagnosis=%s episode_concept_id=32533 AND episode_type_concept_id=32879 LIMIT 1"""
selectEpisodes = "SELECT DISTINCT episode_id FROM episode_event"
curOMOP.execute(selectEpisodes)
episodes = curOMOP.fetchall()
for column in vocabs.keys():
    for episode in episodes:
        curOMOP.execute(selectProcOcurr,(vocabs[column],episode[0]))
        rows=curOMOP.fetchall()
        curOMOP.execute(selectDatePatientEpisode,(episode[0]))
        epiData = curOMOP.fetchall()
        curIDEA.execute(selectIDEAEpisodes,(epiData[0][0],epiData[0][1]))
        idLine = curIDEA.fetchall()
        if rows:
            for row in rows:
                date = row[1]
                value = 1
                insert(column,value,date,columns[column],idLine)
        else:
            value = 0
            insert(column,value,date,columns[column],idLine)

